In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

import glob, os, cv2, time
import numpy as np
from sklearn.model_selection import train_test_split, KFold
os.environ["CUDA_VISIBLE_DEVICES"]='2,3'

BASE_TRAIN_PATH = "./new2_data_sets/train"
BASE_TRAIN_AUGMENTED_PATH = "./new2_data_sets/train_augmented"
SEED = 4444
time_start = time.time()

l_train_files = glob.glob(f'{BASE_TRAIN_PATH}/*orig.png')
l_mask_files = glob.glob(f'{BASE_TRAIN_PATH}/*mask.png')
l_train_aug_files = glob.glob(f'{BASE_TRAIN_AUGMENTED_PATH}/*orig.png')
l_mask_aug_files = glob.glob(f'{BASE_TRAIN_AUGMENTED_PATH}/*mask.png')

img_trains = np.stack([cv2.imread(i,0) for i in l_train_files])
mask_trains = np.stack([cv2.imread(i,0) for i in l_mask_files])
img_trains_aug = np.stack([cv2.imread(i,0) for i in l_train_aug_files])
mask_trains_aug = np.stack([cv2.imread(i,0) for i in l_mask_aug_files])
                
imgs = np.concatenate([img_trains, img_trains_aug], axis=0)
masks = np.concatenate([mask_trains, mask_trains_aug], axis=0)

imgs = imgs / 255.
masks = masks / 255.



print('train_shape:', imgs.shape)
if imgs.shape != masks.shape:
    print('wrong train and mask shape')
    
time_end = time.time()
print("loading times :", time_end - time_start)


train_shape: (8640, 240, 320)
loading times : 389.7957253456116


In [2]:
imgs_train, imgs_val, masks_train, masks_val = train_test_split(imgs, masks, test_size=0.2, shuffle=True, random_state=SEED)
print('imgs_train:',imgs_train.shape, 'imgs_val:', imgs_val.shape, 'masks_train:', masks_train.shape, 'masks_val:', masks_val.shape)

imgs_train: (6912, 240, 320) imgs_val: (1728, 240, 320) masks_train: (6912, 240, 320) masks_val: (1728, 240, 320)


In [8]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        
        CH = 32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=CH, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(CH)
        self.elu1 = nn.ELU()
        self.pool1 = nn.MaxPool2d(2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=CH, out_channels=CH*2, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(CH*2)
        self.elu2 = nn.ELU()
        self.pool2 = nn.MaxPool2d(2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=CH*2, out_channels=CH*4, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(CH*4)
        self.elu3 = nn.ELU()
        self.pool3 = nn.MaxPool2d(2, stride=2)
        
        self.conv4 = nn.Conv2d(in_channels=CH*4, out_channels=CH*8, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(CH*8)
        self.elu4 = nn.ELU()
        self.pool4 = nn.MaxPool2d(2, stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=CH*8, out_channels=CH*16, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(CH*16)
        self.elu5 = nn.ELU()
#         self.pool5 = nn.MaxPool2d(2, stride=2)

        self.conv6 = nn.Conv2d(in_channels=(CH*16+CH*8), out_channels=CH*8, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(CH*8)
        self.elu6 = nn.ELU()
        
        self.conv7 = nn.Conv2d(in_channels=(CH*8+CH*4), out_channels=CH*4, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(CH*4)
        self.elu7 = nn.ELU()
        
        self.conv8 = nn.Conv2d(in_channels=(CH*4+CH*2), out_channels=CH*2, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(CH*2)
        self.elu8 = nn.ELU()
        
        self.conv9 = nn.Conv2d(in_channels=(CH*2+CH), out_channels=CH, kernel_size=3, stride=1, padding=1)
        self.bn9 = nn.BatchNorm2d(CH)
        self.elu9 = nn.ELU()
        
        self.conv10 = nn.Conv2d(in_channels=CH, out_channels=CH, kernel_size=3, stride=1, padding=1)
        self.bn10 = nn.BatchNorm2d(CH)
        self.elu10 = nn.ELU()
        
        
        self.conv11 = nn.Conv2d(in_channels=CH, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input):
        output1 = self.elu1(self.bn1(self.conv1(input)))
        pooled1 = self.pool1(output1)
        output2 = self.elu2(self.bn2(self.conv2(pooled1)))
        pooled2 = self.pool2(output2)
        output3 = self.elu3(self.bn3(self.conv3(pooled2)))
        pooled3 = self.pool3(output3)
        output4 = self.elu4(self.bn4(self.conv4(pooled3)))
        pooled4 = self.pool4(output4)
        output5 = self.elu5(self.bn5(self.conv5(pooled4)))
        
#         output6 = np.concatenate([nn.Upsample(scale_factor=2)(output5),output4],axis=-1)
        output6 = torch.cat([nn.Upsample(scale_factor=2)(output5), output4], dim=1)
        output7 = self.elu6(self.bn6(self.conv6(output6)))
#         output8 = np.concatenate([nn.Upsample(scale_factor=2)(output7),output3],axis=-1)
        output8 = torch.cat([nn.Upsample(scale_factor=2)(output7), output3], dim=1)
        output9 = self.elu7(self.bn7(self.conv7(output8)))
#         output10 = np.concatenate([nn.Upsample(scale_factor=2)(output9),output2],axis=-1)
        output10 = torch.cat([nn.Upsample(scale_factor=2)(output9), output2], dim=1)
        output11 = self.elu8(self.bn8(self.conv8(output10)))
#         output12 = np.concatenate([nn.Upsample(scale_factor=2)(output11),output1],axis=-1)
        output12 = torch.cat([nn.Upsample(scale_factor=2)(output11), output1], dim=1)
        output13 = self.elu9(self.bn9(self.conv9(output12)))
        output14 = self.elu10(self.bn10(self.conv10(output13)))
        output15 = self.sigmoid(self.conv11(output14))

        return output15

    

def dice_score_(img1, img2):
    if not isinstance(img1, numpy.ndarray):
        img1 = np.array(img1)
    if not isinstance(img2, numpy,ndarray):
        img2 = np.array(img2)
    img1_f = img1.reshape(1,-1)
    img2_f = img2.reshape(1,-1)
    intersection = np.sum(img1_f * img2_f)
    
    return (2. * intersection + np.finfo(float).eps) / (np.sum(ing1_f) + np.sum(ing2_f) + np.finfo(float).eps)


def dice_loss_score_Batch(inputs, targets):
    iflat = inputs.view(-1)
    tflat = targets.view(-1)
    
    intersection = (iflat * tflat).sum()
    dice_score = (2. * intersection + 1e-5) / (iflat.sum() + tflat.sum() + 1e-5)
    BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
    dice_loss = BCE + 1-dice_score
    return dice_loss, dice_score

In [47]:
def init_weights(m):
#     classname = m.__class__.__name__
    if isinstance(m, (nn.Conv2d, nn.BatchNorm2d, nn.ELU)):
        torch.nn.init.zeros_(m.bias)
        torch.nn.init.kaiming_uniform_(m.weight)
    
    

In [9]:
imgs_train_expanded = np.expand_dims(imgs_train, -1)
masks_train_expanded = np.expand_dims(masks_train, -1)
imgs_val_expanded = np.expand_dims(imgs_val, -1)
masks_val_expanded = np.expand_dims(masks_val, -1)
print(imgs_train_expanded.shape)

imgs_train_trans = np.transpose(imgs_train_expanded, (0,3,1,2))
masks_train_trans = np.transpose(masks_train_expanded, (0,3,1,2))
imgs_val_trans = np.transpose(imgs_val_expanded, (0,3,1,2))
masks_val_trans = np.transpose(masks_val_expanded, (0,3,1,2))

imgs_train_tensor = torch.tensor(imgs_train_trans, dtype=torch.float32)
masks_train_tensor = torch.tensor(masks_train_trans, dtype=torch.float32)
imgs_val_tensor = torch.tensor(imgs_val_trans, dtype=torch.float32)
masks_val_tensor = torch.tensor(masks_val_trans, dtype=torch.float32)


print(imgs_train_tensor.shape)
print(masks_train_trans.shape)

# imgs_train_tensor = torch.from_numpy(imgs_train_trans).type(torch.float32)
# print(imgs_train_tensor.shape)


(6912, 240, 320, 1)
torch.Size([6912, 1, 240, 320])
(6912, 1, 240, 320)


In [26]:
from torch.utils.data import Dataset, DataLoader


class Dice_BCE_loss(nn.Module):
#     bce = F.binary_cross_entropy_with_logits(pred, target, reduction='sum')
    def __init__(self, weight=None, size_average=True):
        super(Dice_BCE_loss, self).__init__()
        
    def forward(self, inputs, targets, smooth=1e-5):
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)
    
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()
        dice_score = (2. * intersection + 1e-5)/(inputs.sum() + targets.sum() + 1e-5)
        dice_loss = 1 - dice_score
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
    
        loss = bce + dice_loss
    
        return loss, dice_score


class MyDataset(Dataset):
    def __init__(self, imgs, masks, dtype=torch.float32):
        super(MyDataset,self).__init__()
        self.imgs = imgs
        self.masks = masks
        if not torch.is_tensor(self.imgs):
            self.imgs = torch.tensor(self.imgs, dtype=torch.float32)
        if not torch.is_tensor(self.masks):
            self.masks = torch.tensor(self.masks, dtype=torch.float32)
        assert(len(self.imgs) == len(self.masks))
            
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        return {'img': self.imgs[idx], 'mask': self.masks[idx]}
    
    
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for i_batch, sample_batched in enumerate(dataloader):
    #     if i_batch == 1:

    #         print(len(sample_batched['img']))
    #         print(sample_batched['img'].shape)
    #         print(sample_batched['mask'].shape)
    #     if i_batch > max_ibatch:
    #         max_ibatch = i_batch
        train_start_time = time.time()
        X = sample_batched['img'].to('cuda')
        y = sample_batched['mask'].to('cuda')

        pred = model(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i_batch % 200 == 0:
            loss, current = loss.item(), i_batch * len(X)
            d_loss, d_score = dice_loss_score_Batch(pred,y)
            print(f"loss: {loss:.5f}, dice_loss: {d_loss:.5f}, dice_score: {d_score:.5f}  [{current:>4d}/{size:>4d}],time: {time.time()-train_start_time:.5f}")
            
            
def test_loop(dataloader, model, loss_fn):
    global g_dice_score_list
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    test_dice_loss, test_dice_score = 0, 0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(dataloader):
            X = sample_batched['img'].to('cuda')
            y = sample_batched['mask'].to('cuda')
            pred = model(X)
            d_loss, d_score = dice_loss_score_Batch(pred,y)
            
            test_dice_loss += d_loss
            test_dice_score += d_score
            test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_dice_loss /= num_batches
    test_dice_score /= num_batches
    test_loss /= num_batches
    g_dice_score_list.append(test_dice_score)
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Ave loss: {test_loss:>8f} \n")
    print(f"Test loss: \n loss: {test_loss:.5f}, dice_loss: {test_dice_loss:.5f}, dice_score: {test_dice_score:.5f}")

    
    


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EPOCH = 70
BATCH_SIZE = 5


model = Network().to(device)
# model.apply(init_weights)
loss_fn = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

train_dataset = MyDataset(imgs_train_tensor, masks_train_trans, dtype=torch.float32)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = MyDataset(imgs_val_tensor, masks_val_tensor, dtype=torch.float32)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

total_time = time.time()
global g_dice_score_list
g_dice_score_list = []

for t in range(EPOCH):
    epoch_start_time = time.time()
    print(f"Epoch {t+1}\n----------------------------")
    
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(val_dataloader, model, loss_fn)
    print(f"Epoch {t+1} done, time: {time.time()-epoch_start_time:.5f}")
    
print(f"Done!  time:{time.time()-total_time:.5f}")


ValueError: Fan in and fan out can not be computed for tensor with fewer than 2 dimensions

In [36]:

# sm = torch.jit.script(model)
# sm.save("scripted_model.pt")

MODEL_HEIGHT = 240
MODEL_WIDTH = 320

average_dice_score = sum(g_dice_score_list)/len(g_dice_score_list)
# l_ = [ i.item()[0] for i in g_dice_score_list]
a = torch.tensor(g_dice_score_list)
print(a)
print(f"average_dice_score : {average_dice_score}")
# example = torch.rand(1,1,MODEL_HEIGHT,MODEL_WIDTH)
# traced_script_module = torch.jit.trace(model, example.to('cuda'))
# traced_script_module.save(f"traced_model_E{EPOCH}_B{BATCH_SIZE}_R{SEED}_S{average_dice_score}.pt")

tensor([0.8100, 0.8650, 0.9086, 0.9182, 0.9232, 0.9248, 0.9226, 0.9254, 0.9366,
        0.9252, 0.9214, 0.9313, 0.9275, 0.9303, 0.9334, 0.9335, 0.9273, 0.9292,
        0.9335, 0.9375, 0.9337, 0.9368, 0.9290, 0.9360, 0.9368, 0.9313, 0.9335,
        0.9334, 0.9345, 0.9274, 0.9289, 0.9223, 0.9321, 0.9038, 0.9287, 0.9379,
        0.9335, 0.9087, 0.9311, 0.9393, 0.9095, 0.9336, 0.9228, 0.9349, 0.9395,
        0.9337, 0.9291, 0.9328, 0.9347, 0.9319, 0.9361, 0.9323, 0.9246, 0.9238,
        0.9334, 0.9216, 0.9370, 0.9340, 0.9290, 0.9373, 0.9069, 0.9355, 0.9257,
        0.9377, 0.9324, 0.9371, 0.9268, 0.9307, 0.9267, 0.9323])
average_dice_score : 0.9266654849052429


In [51]:
# params = [i for i in model.parameters()]
params = list(model.parameters())
print(type(params[0]))
print(params[0].grad)

<class 'torch.nn.parameter.Parameter'>
None


In [24]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
Network                                  --
├─Conv2d: 1-1                            320
├─BatchNorm2d: 1-2                       64
├─ELU: 1-3                               --
├─MaxPool2d: 1-4                         --
├─Conv2d: 1-5                            18,496
├─BatchNorm2d: 1-6                       128
├─ELU: 1-7                               --
├─MaxPool2d: 1-8                         --
├─Conv2d: 1-9                            73,856
├─BatchNorm2d: 1-10                      256
├─ELU: 1-11                              --
├─MaxPool2d: 1-12                        --
├─Conv2d: 1-13                           295,168
├─BatchNorm2d: 1-14                      512
├─ELU: 1-15                              --
├─MaxPool2d: 1-16                        --
├─Conv2d: 1-17                           1,180,160
├─BatchNorm2d: 1-18                      1,024
├─ELU: 1-19                              --
├─Conv2d: 1-20                           1,7

In [ ]:
tensor([0.8100, 0.8650, 0.9086, 0.9182, 0.9232, 0.9248, 0.9226, 0.9254, 0.9366,
        0.9252, 0.9214, 0.9313, 0.9275, 0.9303, 0.9334, 0.9335, 0.9273, 0.9292,
        0.9335, 0.9375, 0.9337, 0.9368, 0.9290, 0.9360, 0.9368, 0.9313, 0.9335,
        0.9334, 0.9345, 0.9274, 0.9289, 0.9223, 0.9321, 0.9038, 0.9287, 0.9379,
        0.9335, 0.9087, 0.9311, 0.9393, 0.9095, 0.9336, 0.9228, 0.9349, 0.9395,
        0.9337, 0.9291, 0.9328, 0.9347, 0.9319, 0.9361, 0.9323, 0.9246, 0.9238,
        0.9334, 0.9216, 0.9370, 0.9340, 0.9290, 0.9373, 0.9069, 0.9355, 0.9257,
        0.9377, 0.9324, 0.9371, 0.9268, 0.9307, 0.9267, 0.9323])
average_dice_score : 0.9266654849052429